# Data

In [1]:
from src.mtal.data_collect import get_spot_pairs

cryptos = get_spot_pairs()

In [2]:
from src.mtal.data_collect import get_pair_df

# df = get_pair_df(pair="BTCUSDT")
df = get_pair_df(pair="BTCUSDT", limit=900, frequency="1d")
df.head()

,Open Time,Open,High,Low,Close,Volume,Close Time
0,2021-11-12,64774.25,65450.70,62278.00,64122.23,44490.10816,2021-11-12 23:59:59.999
1,2021-11-13,64122.22,65000.00,63360.22,64380.00,22504.97383,2021-11-13 23:59:59.999
2,2021-11-14,64380.01,65550.51,63576.27,65519.10,25705.07347,2021-11-14 23:59:59.999
3,2021-11-15,65519.11,66401.82,63400.00,63606.74,37829.37124,2021-11-15 23:59:59.999
4,2021-11-16,63606.73,63617.31,58574.07,60058.87,77455.15609,2021-11-16 23:59:59.999


# EMA backtest

In [3]:
from src.mtal.utils import generate_pinescript
from src.mtal.backtesting.ma_cross_backtest import MACrossBacktester 

short_ema = 2
long_ema = 4
tester = MACrossBacktester(
    short_ma=short_ema, long_ma=long_ema, data=df
)
results = tester.run()

entry_dates = [int(entry_date.timestamp() * 1000) for entry_date in results.entry_dates]
exit_dates = [int(exit_date.timestamp() * 1000) for exit_date in results.exit_dates]

print(results)
print(generate_pinescript(entry_dates, exit_dates))


BacktestResults(pnl=198.0874294785674, pnl_percentage=0.1980874294785674, max_drawdown=-0.09294138090824834, win_rate=0.3010752688172043, average_return=0.00484610621416235, trade_number=93, entry_dates=[Timestamp('2021-11-14 00:00:00'), Timestamp('2021-11-26 00:00:00'), Timestamp('2021-11-29 00:00:00'), Timestamp('2021-12-13 00:00:00'), Timestamp('2021-12-16 00:00:00'), Timestamp('2021-12-22 00:00:00'), Timestamp('2022-01-12 00:00:00'), Timestamp('2022-01-28 00:00:00'), Timestamp('2022-02-05 00:00:00'), Timestamp('2022-02-16 00:00:00'), Timestamp('2022-02-26 00:00:00'), Timestamp('2022-03-01 00:00:00'), Timestamp('2022-03-10 00:00:00'), Timestamp('2022-03-15 00:00:00'), Timestamp('2022-03-23 00:00:00'), Timestamp('2022-04-04 00:00:00'), Timestamp('2022-04-19 00:00:00'), Timestamp('2022-04-26 00:00:00'), Timestamp('2022-04-29 00:00:00'), Timestamp('2022-05-05 00:00:00'), Timestamp('2022-05-16 00:00:00'), Timestamp('2022-05-18 00:00:00'), Timestamp('2022-05-20 00:00:00'), Timestamp('202

In [4]:
from itertools import combinations
from src.mtal.backtesting.ma_cross_backtest import MACrossBacktester


ema_values = range(1, 60)  

combinations = [(short_ema, long_ema) for short_ema, long_ema in combinations(ema_values, 2)]

results = {}

for short_ema, long_ema in combinations:
    tester = MACrossBacktester(short_ma=short_ema, long_ma=long_ema, data=df)
    test_results = tester.run()
    results[(short_ema, long_ema)] = test_results

best_combination = max(results, key=lambda x: results[x].pnl)  # Ajuster le critère si nécessaire
best_result = results[best_combination]

print(f"Meilleure combinaison: Short EMA = {best_combination[0]}, Long EMA = {best_combination[1]}")
print(f"Résultat du test: {best_result}")


Meilleure combinaison: Short EMA = 56, Long EMA = 57
Résultat du test: BacktestResults(pnl=1501.0192927247012, pnl_percentage=1.5010192927247012, max_drawdown=-0.06711928128001221, win_rate=0.6666666666666666, average_return=0.439401610675707, trade_number=3, entry_dates=[Timestamp('2021-11-14 00:00:00'), Timestamp('2023-01-19 00:00:00'), Timestamp('2023-10-22 00:00:00')], exit_dates=[Timestamp('2021-11-17 00:00:00'), Timestamp('2023-08-22 00:00:00'), Timestamp('2024-04-28 00:00:00')], entry_prices=[64380.01, 20677.47, 29909.8], exit_prices=[60058.87, 26126.92, 63461.98], profit_pct_history=[-0.06711928128001221, 0.26354529833678864, 1.1217788149703445], profit_history=[-67.11928128001222, 245.856327327697, 1322.2822466770165])


# VWMA Backtest

In [5]:
from src.mtal.utils import generate_pinescript
from src.mtal.backtesting.ma_cross_backtest import MACrossBacktester 

short_ema = 51
long_ema = 57

tester = MACrossBacktester(short_ma=short_ema, long_ma=long_ema, data=df, ma_type="vwma"
)
results = tester.run()

entry_dates = [int(entry_date.timestamp() * 1000) for entry_date in results.entry_dates]
exit_dates = [int(exit_date.timestamp() * 1000) for exit_date in results.exit_dates]

print(results)
# print(generate_pinescript(entry_dates, exit_dates))



BacktestResults(pnl=1870.5664822175513, pnl_percentage=1.8705664822175514, max_drawdown=-0.2109119179358869, win_rate=0.45454545454545453, average_return=0.1323731399829349, trade_number=11, entry_dates=[Timestamp('2022-03-15 00:00:00'), Timestamp('2022-04-13 00:00:00'), Timestamp('2022-04-27 00:00:00'), Timestamp('2022-08-09 00:00:00'), Timestamp('2022-10-28 00:00:00'), Timestamp('2022-11-12 00:00:00'), Timestamp('2023-01-05 00:00:00'), Timestamp('2023-01-07 00:00:00'), Timestamp('2023-07-03 00:00:00'), Timestamp('2023-10-12 00:00:00'), Timestamp('2024-02-01 00:00:00')], exit_dates=[Timestamp('2022-04-01 00:00:00'), Timestamp('2022-04-24 00:00:00'), Timestamp('2022-05-10 00:00:00'), Timestamp('2022-09-10 00:00:00'), Timestamp('2022-10-31 00:00:00'), Timestamp('2022-11-18 00:00:00'), Timestamp('2023-01-06 00:00:00'), Timestamp('2023-06-01 00:00:00'), Timestamp('2023-08-17 00:00:00'), Timestamp('2024-01-30 00:00:00'), Timestamp('2024-04-28 00:00:00')], entry_prices=[39671.37, 40074.95, 

In [6]:
from itertools import combinations
from src.mtal.backtesting.ma_cross_backtest import MACrossBacktester


ema_values = range(1, 60)  

combinations = [(short_ema, long_ema) for short_ema, long_ema in combinations(ema_values, 2)]

results = {}

for short_ema, long_ema in combinations:
    tester = MACrossBacktester(short_ma=short_ema, long_ma=long_ema, data=df, ma_type="vwma")
    test_results = tester.run()
    results[(short_ema, long_ema)] = test_results

best_combination = max(results, key=lambda x: results[x].pnl)  # Ajuster le critère si nécessaire
best_result = results[best_combination]

print(f"Meilleure combinaison: Short EMA = {best_combination[0]}, Long EMA = {best_combination[1]}")
print(f"Résultat du test: {best_result}")


/home/romain/mtal/src/mtal/analysis.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[get_ma_names(span, "vwma")] = vwma
/home/romain/mtal/src/mtal/analysis.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[get_ma_names(span, "vwma")] = vwma
/home/romain/mtal/src/mtal/analysis.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragment

Meilleure combinaison: Short EMA = 51, Long EMA = 57
Résultat du test: BacktestResults(pnl=1870.5664822175513, pnl_percentage=1.8705664822175514, max_drawdown=-0.2109119179358869, win_rate=0.45454545454545453, average_return=0.1323731399829349, trade_number=11, entry_dates=[Timestamp('2022-03-15 00:00:00'), Timestamp('2022-04-13 00:00:00'), Timestamp('2022-04-27 00:00:00'), Timestamp('2022-08-09 00:00:00'), Timestamp('2022-10-28 00:00:00'), Timestamp('2022-11-12 00:00:00'), Timestamp('2023-01-05 00:00:00'), Timestamp('2023-01-07 00:00:00'), Timestamp('2023-07-03 00:00:00'), Timestamp('2023-10-12 00:00:00'), Timestamp('2024-02-01 00:00:00')], exit_dates=[Timestamp('2022-04-01 00:00:00'), Timestamp('2022-04-24 00:00:00'), Timestamp('2022-05-10 00:00:00'), Timestamp('2022-09-10 00:00:00'), Timestamp('2022-10-31 00:00:00'), Timestamp('2022-11-18 00:00:00'), Timestamp('2023-01-06 00:00:00'), Timestamp('2023-06-01 00:00:00'), Timestamp('2023-08-17 00:00:00'), Timestamp('2024-01-30 00:00:00')

ema: 2, 4 daily -> 56, 57
VWMA: 51, 57